# Tools for running QVT

Using qiskit 0.20.0

In [1]:
import numpy as np
from qiskit.ignis.verification import qv_circuits
import qiskit.ignis.verification.quantum_volume as qv
from qvwrappers.error_models import arbitrary_noise
from qiskit import Aer, execute

nqubits = 6
ntrials = 100
nshots = 100

## Make circuits

In [2]:
qc, qc_no_meas = qv_circuits([list(range(nqubits))], ntrials)

## Ideal simulation

In [3]:
backend = Aer.get_backend('statevector_simulator')
ideal_results = execute(
    [qc_no_meas[i][0] for i in range(len(qc_no_meas))], 
    backend=backend
).result()

## Noisy simulation

In [5]:
from error_models import arbitrary_noise
from transpiler_passes import preset_passes

pm = preset_passes('high', {'tol': 0.996, 'mirror': True})
qc_opt = [pm.run(qc[i][0]) for i in range(len(qc))]

noise_model = arbitrary_noise(
    {'tq_dep': 1e-2, 'sq_dep': 5e-5, 'meas': 3e-3}, 
    nqubits
)

backend = Aer.get_backend('qasm_simulator')

sim_results = execute(
    qc_opt, 
    backend=backend,
    optimization_level=0,
    shots=nshots,
    noise_model=noise_model
).result()

ImportError: attempted relative import with no known parent package

## Analysis

In [10]:
qv_fitter = qv.QVFitter(qubit_lists=[list(range(nqubits))])
qv_fitter.add_statevectors(ideal_results)
qv_fitter.add_data(sim_results)

print(f'Measured heavy output frequency = {qv_fitter.ydata[0][0]}')

Measured heavy output frequency = 0.7292


## Confidence intervals

In [8]:
from analysis_util import success_lower, bootstrap

original_lower = success_lower(qv_fitter.ydata[0][0], ntrials)
original_pass = 'pass' if (original_lower > 2/3) else 'fail'

bootstrap_sample = bootstrap(qv_fitter)
bootstrap_lower = 2 * qv_fitter.ydata[0][0] - np.quantile(bootstrap_sample, 0.9773)
bootstrap_pass = 'pass' if (bootstrap_lower > 2/3) else 'fail'

print(f'Original method lower bound  = {original_lower:.4f} ({original_pass})')
print(f'Bootstrap method lower bound = {bootstrap_lower:.4f} ({bootstrap_pass})')

Original method lower bound  = 0.6403 (fail)
Bootstrap method lower bound = 0.7161 (pass)
